In [ ]:
import cv2
import numpy as np
from deepface import DeepFace
import time
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime

# Base de données étudiants liée à chaque table (table 1 => étudiant 1, etc)
database = {
    1: {"nom": "Haddaoui", "prenom": "Soukaina", "email": "soukainahaddaoui@gmail.com", "photo": "haddaoui_soukaina.jpg"},
    2: {"nom": "Fakhi", "prenom": "Chaimaa", "email": "sheimaafakhietudes@gmail.com", "photo": "Fakhi_chaimaa.jpg"},
    3: {"nom": "Elkhale", "prenom": "Abdesslam", "email": "abdessalam_elkhale@um5.ac.ma", "photo": "Elkhale_Abdesslam.jpg"},
}

EMAIL_COORDINATEUR = "sheimaafakhietudes@gmail.com"
EMAIL_SENDER = "haddaouisoukaina7@gmail.com"  # Remplacer par ton mail
EMAIL_PASSWORD = "egsy kavn fbec kehv"         # Mot de passe ou mot de passe d’application

def send_email_fraude(etudiant):
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    sujet = "Alerte fraude détectée"
    corps = f"""
    Bonjour,

    Une tentative de fraude a été détectée.

    Étudiant concerné :
    Nom : {etudiant['nom']}
    Prénom : {etudiant['prenom']}
    Email étudiant : {etudiant['email']}

    Date et heure de détection : {now}

    Merci de prendre les mesures nécessaires.

    Cordialement,
    Système de surveillance ENSAM
    """

    msg = MIMEMultipart()
    msg['From'] = EMAIL_SENDER
    msg['To'] = EMAIL_COORDINATEUR
    msg['Subject'] = sujet
    msg.attach(MIMEText(corps, 'plain'))

    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.send_message(msg)
        print(f"Email envoyé au coordinateur pour l'étudiant {etudiant['prenom']} {etudiant['nom']}.")
    except Exception as e:
        print(f"Erreur lors de l'envoi de l'email : {e}")

def take_photo(filename='photo.jpg', quality=0.8):
    try:
        js = Javascript('''
        async function takePhoto(quality) {
          const div = document.createElement('div');
          const video = document.createElement('video');
          const button = document.createElement('button');
          button.textContent = 'Prendre photo';
          div.appendChild(video);
          div.appendChild(button);
          document.body.appendChild(div);

          const stream = await navigator.mediaDevices.getUserMedia({video: true});
          video.srcObject = stream;
          await video.play();

          google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

          await new Promise((resolve) => button.onclick = resolve);

          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video, 0, 0);

          stream.getVideoTracks()[0].stop();
          div.remove();

          return canvas.toDataURL('image/jpeg', quality);
        }
        ''')
        display(js)
        data = eval_js(f'takePhoto({quality})')
        binary = b64decode(data.split(',')[1])
        with open(filename, 'wb') as f:
            f.write(binary)
        return filename
    except Exception as e:
        print(f"Erreur lors de la capture photo : {e}")
        return None

def detect_red_led(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_red1 = np.array([0, 120, 70])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 120, 70])
    upper_red2 = np.array([180, 255, 255])
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = mask1 + mask2
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        if cv2.contourArea(cnt) > 50:
            return True
    return False

def detect_blue_led(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_blue = np.array([100, 150, 50])
    upper_blue = np.array([140, 255, 255])
    mask = cv2.inRange(hsv, lower_blue, upper_blue)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        if cv2.contourArea(cnt) > 50:
            return True
    return False

def identify_student(photo_path):
    # Parcours toute la base et retourne le premier étudiant reconnu ou None
    for idx, etudiant in database.items():
        try:
            print(f"Comparaison avec {etudiant['nom']} (photo: {etudiant['photo']})...")
            result = DeepFace.verify(photo_path, etudiant["photo"], enforce_detection=False)
            if result["verified"]:
                print(f"✅ Étudiant reconnu : {etudiant['prenom']} {etudiant['nom']}")
                return etudiant
        except Exception as e:
            print(f"Erreur reconnaissance faciale avec {etudiant['nom']}: {e}")
    return None

def main_loop():
    total_tables = len(database)
    table_index = 1

    while table_index <= total_tables:
        print(f"\n--- Table {table_index} ---")
        print("Prendre photo de la table (LED)")

        table_photo = take_photo(filename=f'table_{table_index}.jpg')
        if table_photo is None:
            print("Erreur lors de la capture de la table.")
            break

        frame = cv2.imread(table_photo)
        if frame is None:
            print("Impossible de lire la photo de la table.")
            continue

        blue_detected = detect_blue_led(frame)
        red_detected = detect_red_led(frame)

        if blue_detected:
            print("LED bleue détectée → Aucune tentative de fraude détectée. Passage à la table suivante.")
            table_index += 1
            time.sleep(2)

        elif red_detected and not blue_detected:
            print("LED rouge détectée → capture visage étudiant.")
            face_photo = take_photo(filename=f'visage_{table_index}.jpg')
            if face_photo is None:
                print("Erreur lors de la capture du visage.")
                break

            etudiant = identify_student(face_photo)
            if etudiant:
                print("✅ Étudiant reconnu :")
                for k, v in etudiant.items():
                    if k != "photo":
                        print(f"{k.capitalize()} : {v}")
                send_email_fraude(etudiant)
            else:
                print("❌ Aucun étudiant reconnu.")
            table_index += 1
            time.sleep(2)

        else:
            print("Aucune LED détectée. Merci de reprendre la photo de la table.")
            # On ne passe pas à la table suivante, on refait la photo

    print("\nFin du traitement des tables.")

if __name__ == "__main__":
    main_loop()




--- Table 1 ---
Prendre photo de la table (LED)


<IPython.core.display.Javascript object>

LED rouge détectée → capture visage étudiant.


<IPython.core.display.Javascript object>

Comparaison avec Haddaoui (photo: haddaoui_soukaina.jpg)...
✅ Étudiant reconnu : Soukaina Haddaoui
✅ Étudiant reconnu :
Nom : Haddaoui
Prenom : Soukaina
Email : soukainahaddaoui@gmail.com
Email envoyé au coordinateur pour l'étudiant Soukaina Haddaoui.

--- Table 2 ---
Prendre photo de la table (LED)


...

LED bleue détectée → Aucune tentative de fraude détectée. Passage à la table suivante.

--- Table 3 ---
Prendre photo de la table (LED)


<IPython.core.display.Javascript object>

LED rouge détectée → capture visage étudiant.


<IPython.core.display.Javascript object>

Comparaison avec Haddaoui (photo: haddaoui_soukaina.jpg)...
Comparaison avec Fakhi (photo: Fakhi_chaimaa.jpg)...
✅ Étudiant reconnu : Chaimaa Fakhi
✅ Étudiant reconnu :
Nom : Fakhi
Prenom : Chaimaa
Email : sheimaafakhietudes@gmail.com
Email envoyé au coordinateur pour l'étudiant Chaimaa Fakhi.

Fin du traitement des tables.
